# Generadores y comprensiones
Antes de tocar este nuevo tema, vamos a revisar un punto clave con respecto a la creación de funciones que en la sesión anterior no se colocó, nos referimos a las anotaciones en funciones o function annotations.

Python nos permite añadir el tipo de los argumentos de entrada y salida de una función. A continuación podemos ver un ejemplo con la función rest(), que recibe dos argumentos a, b y cuyo tipo se espera que sea int.

In [1]:
def rest(a: int, b: int) -> int:
    return a - b

print(f"{rest(9, 6) = }")

rest(9, 6) = 3


Sin embargo es muy importante notar que Python ignora las anotaciones. Es decir, son una mera nota en el código que indica el tipo esperado, pero el siguiente código no daría ningún error:

In [2]:
print(f"{rest(7.2, 3.2) = }")

rest(7.2, 3.2) = 4.0


Hay que entender que Python es un lenguaje de programación con tipado dinámico y duck typing, lo que significa que los tipos (int, string, etc) le dan igual. Precisamente esto es lo que hace que el siguiente código funcione. La función imprime puede ser llamada con cualquier tipo, ya que Python no realiza ninguna comprobación del tipo de var.

In [2]:
def imprime(var):
    print(var)

imprime(7.0)
imprime(10)
imprime("Hello, Python")
imprime(False)

7.0
10
Hello, Python
False


Sin embargo, en ciertas ocasiones esto nos puede traer problemas. ¿Y si queremos que la función imprime sólo acepte que var sea de un tipo concreto? Pues bien, las anotaciones en funciones o function annotations como acabamos de ver nos permiten especificar los tipos que se esperan recibir.

Tener en cuenta que las anotaciones en funciones no definen per se una semántica propia. Es decir, podemos escribir lo que se nos ocurra después de cada argumento. Las anotaciones pueden ser accedidas usando `__annotations__`.

Veremos cuatro ejemplos; en el primero de ellos, colocamos cualquier anotación en un argumento; en el segundo caso, usaremos los tipos de datos que tienen integrado Python; en el tercer caso, usaremos una anotación en clases definidas por el usuario y en el último caso, asignameros una anotación a una variable que creemos:

In [6]:
# Primer caso
'''
def suma(a: 'parametro 1', b: 'parametro 2') -> 'retorno':
    return a + b

print(suma.__annotations__)
'''

# Segundo caso
def filtrar_pares(salida: 'list' = []) -> 'list':
    return [i for i in salida if i%2 == 0]

print(filtrar_pares([1, 2, 3, 4, 5, 6]))

# Tercer caso
class PrimeraClase:
    pass

def funcion(a: PrimeraClase) -> PrimeraClase:
    return a

a = PrimeraClase()
funcion(a)

# Cuarto caso
pi: float = 3.14
print(pi)

[2, 4, 6]
3.14


Como se menciono antes, Python ignora las anotaciones por lo cual no daría error si colocamos un tipo de dato erróneo. Para evitar ello, podemos usar el siguiente método (tener en cuenta que no es muy recomendado):

In [5]:
def suma(a: int, b: int) -> int:
    if isinstance(a, suma.__annotations__['a']) and isinstance(b, suma.__annotations__['b']):
        return a + b
    else:
        raise Exception("Error de tipos")

print(suma(7, 3))
try:
    print(suma(7.0, 3.0))
except Exception as e:
    print(e)


10
Error de tipos


Afortunadamente, existen herramientas como *mypy* que nos permiten hacer un chequeo estático de los tipos, obteniendo el error antes de que el código se ejecute. Lo podemos instalar de la siguiente manera en la terminal: *pip instal mypy*

Retomando con la sesión de hoy, a veces, hay razones válidas para no llevar nuestro código hasta el límite máximo: por ejemplo, a veces, para conseguir una mejora insignificante, tenemos que sacrificar la legibilidad o la mantenibilidad. ¿Tiene algún sentido servir una página web en 1 segundo con código ilegible y complicado, cuando podemos servirla en 1,05 segundos con código legible y limpio? No, no tiene sentido.

Por otro lado, a veces es perfectamente razonable tratar de recortar un milisegundo de una función, especialmente cuando la función está destinada a ser llamada miles de veces. Cada milisegundo que ahorres ahí significa segundos ahorrados en miles de llamadas, y esto podría ser significativo para tu aplicación.

A la luz de estas consideraciones, el enfoque de esta sesión no será darte las herramientas para llevar tu código a los límites absolutos de rendimiento y optimización sin importar qué, sino más bien permitirte escribir código eficiente y elegante que se lea bien, se ejecute rápido y no desperdicie recursos de manera obvia. Para ello, vamos a cubrir los siguientes temas:
- [Las funciones *map()*, *zip()* y *filter()*]()
- [Comprensiones]()
- [Generadores]()

Realizaremos varias mediciones y comparaciones y, con cautela, sacaremos algunas conclusiones. Vamos a echar un vistazo al siguiente código:


In [ ]:
def square1(n):
    return n ** 2 # squaring through the power operator
def square2(n):
    return n * n # squaring through multiplication

Ambas funciones devuelven el cuadrado de *n*, pero ¿cuál es más rápida? A partir de una sencilla prueba comparativa, parece que la segunda es ligeramente más rápida. Si lo piensas, tiene sentido: calcular la potencia de un número implica multiplicar y, por tanto, sea cual sea el algoritmo que utilices para realizar la operación de potencia, no es probable que supere a una multiplicación simple como la de *square2*.

¿Nos importa este resultado? En la mayoría de los casos, no. Si estás programando un sitio web de comercio electrónico, lo más probable es que nunca necesites elevar un número a la segunda potencia, y si lo haces, es probable que sea una operación esporádica. No tiene por qué preocuparse de ahorrar una fracción de microsegundo en una función que llamará unas pocas veces.

Entonces, ¿cuándo es importante la optimización? Un caso muy común es cuando tienes que tratar con enormes colecciones de datos. Si estás aplicando la misma función a un millón de objetos cliente, entonces querrás que tu función esté afinada al máximo. Ganar una décima de segundo en una función llamada un millón de veces le ahorra 100.000 segundos, lo que equivale a unas 27.7 horas: ¡es una gran diferencia! Así que, centrémonos en las colecciones, y veamos qué herramientas te da Python para manejarlas con eficiencia y gracia.

Algunos de los objetos que vamos a explorar son iteradores, que ahorran memoria al operar sobre un único elemento de una colección cada vez en lugar de crear una copia modificada. Como resultado, se necesita algo de trabajo extra si sólo queremos mostrar el resultado de la operación. A menudo recurriremos a envolver el iterador en un constructor *list()*. Esto se debe a que pasar un iterador a *list(...)* lo agota y pone todos los elementos generados en una lista recién creada, que podemos imprimir fácilmente para mostrar su contenido. Veamos un ejemplo de uso de la técnica sobre un objeto range:

In [11]:
print(f"{range(7) = }")
print(f"{list(range(7)) = }")

range(7) = range(0, 7)
list(range(7)) = [0, 1, 2, 3, 4, 5, 6]


Al escribir *range(7)* en una consola Python no muestra el contenido del rango, porque en realidad *range* nunca carga toda la secuencia de números en memoria. Por ello, si envolvemos el rango en una *list()* nos permite ver los números que ha generado.

# Funciones map, zip y filter
Comenzaremos revisando *`map()`*, *`filter()`* y *`zip()`*, que son las principales funciones incorporadas que puedes emplear cuando manejas colecciones, y luego aprenderemos cómo lograr los mismos resultados utilizando dos construcciones muy importantes: comprensiones y generadores.

## map
Según la documentación oficial de Python, *`map(function, iterable, ...)`*, devuelve un iterador que aplica la función a cada elemento del iterable, obteniendo los resultados. Si se pasan argumentos adicionales al iterable, la función debe tomar ese número de argumentos y se aplica a los elementos de todos los iterables en paralelo. Con múltiples iterables, el iterador se detiene cuando se agota el iterable más corto.

Esto se puede traducir en código de la siguiente manera como ejemplo, para ello vamos a usar una función lambda que toma un número variable de argumentos posicionales y los devuelve como una tupla:

In [15]:
print(f"{map(lambda *a: a, range(3)) = }")
print(f"{list(map(lambda *a: a, range(3))) = }") # 1 iterable
print(f"{list(map(lambda *a: a, range(3), 'abc')) = }") # 2 iterable
print(f"{list(map(lambda *a: a, range(3), 'abc', range(4, 7))) = }") # 3 iterable
print(f"{list(map(lambda *a: a, (), 'abc')) = }") # map stops at the shortest iterator (empty tuple is shortest)
print(f"{list(map(lambda *a: a, (1, 2), 'abc')) = }") # map stops at the shortest iterator ((1, 2) shortest)
print(f"{list(map(lambda *a: a, (1, 2, 3, 4), 'abc')) = }") # map stops at the shortest iterator ('abc' shortest)


map(lambda *a: a, range(3)) = <map object at 0x000002015B54AA40>
list(map(lambda *a: a, range(3))) = [(0,), (1,), (2,)]
list(map(lambda *a: a, range(3), 'abc')) = [(0, 'a'), (1, 'b'), (2, 'c')]
list(map(lambda *a: a, range(3), 'abc', range(4, 7))) = [(0, 'a', 4), (1, 'b', 5), (2, 'c', 6)]
list(map(lambda *a: a, (), 'abc')) = []
list(map(lambda *a: a, (1, 2), 'abc')) = [(1, 'a'), (2, 'b')]
list(map(lambda *a: a, (1, 2, 3, 4), 'abc')) = [(1, 'a'), (2, 'b'), (3, 'c')]


En el código anterior, se puede ver por qué tenemos que envolver las llamadas en *list(...)*. Sin ella, obtenemos la representación en cadena de un objeto *map*, lo que no es realmente útil en este contexto.

También puedes notar cómo los elementos de cada iterable se aplican a la función; al principio, el primer elemento de cada iterable, luego el segundo de cada iterable, y así sucesivamente. Observa también que *map()* se detiene cuando se agota el más corto de los iterables con los que lo hemos llamado. Este es realmente un comportamiento muy bueno; no nos obliga a nivelar todos los iterables a una longitud común, ni se rompe si no son todos de la misma longitud. 

*map()* es muy útil cuando tienes que aplicar la misma función a una o más colecciones de objetos. Veremos un ejemplo más a detalle: queremos ordenar de manera descendente por la suma de créditos acumulados por los alumnos, de forma que el mejor alumno quede en la posición 0; para ello, se va a escribir una función para producir un objeto transformado, lo ordenamos y, a continuación, deshacemos la ordenación; en la cual, cada alumno tiene créditos en tres asignaturas (posiblemente diferentes); luego de ello, revertimos los objetos decorados o transformados para obtener de ellos los originales.

In [48]:
students = [
    dict(id=0, credits=dict(math=9, physics=6, history=7)),
    dict(id=1, credits=dict(math=6, physics=7, latin=10)),
    dict(id=2, credits=dict(history=8, physics=9, chemistry=10)),
    dict(id=3, credits=dict(math=5, physics=5, geography=7)),
]

def decorate(student):
    # creamos 2-tuple (sum of credits, student) para el dict de students
    return (sum(student["credits"].values()), student)

def undecorate(decorated_student):
    # Descartar la suma de créditos y devuelve el dict original
    return decorated_student[1]

students_decorate = sorted(map(decorate, students), reverse=True)
students_undecorate = list(map(undecorate, students_decorate))
print(f"{students_decorate = }")
print(f"{students_undecorate = }")

students_decorate = [(27, {'id': 2, 'credits': {'history': 8, 'physics': 9, 'chemistry': 10}}), (23, {'id': 1, 'credits': {'math': 6, 'physics': 7, 'latin': 10}}), (22, {'id': 0, 'credits': {'math': 9, 'physics': 6, 'history': 7}}), (17, {'id': 3, 'credits': {'math': 5, 'physics': 5, 'geography': 7}})]
students_undecorate = [{'id': 2, 'credits': {'history': 8, 'physics': 9, 'chemistry': 10}}, {'id': 1, 'credits': {'math': 6, 'physics': 7, 'latin': 10}}, {'id': 0, 'credits': {'math': 9, 'physics': 6, 'history': 7}}, {'id': 3, 'credits': {'math': 5, 'physics': 5, 'geography': 7}}]


La variable *students* es un diccionario con dos claves: *id* y *credits*. El clave de créditos también es un diccionario en el que hay tres pares clave/valor asignatura/grado. Al usar *dict.values()* devuelve un objeto iterable, con sólo los valores del diccionario. Por lo tanto, *sum(student['créditos'].values())* para el primer alumno es equivalente a *sum((9, 6, 7))*.

Podemos ordenar por su número total de créditos con sólo ordenar la lista de tuplas. Para aplicar la decoración a cada elemento de estudiantes, llamamos a map(decorar, estudiantes). Ordenamos el resultado, y luego deshacemos la decoración de forma similar.

Una cosa que hay que tener en cuenta sobre la parte de ordenación es lo que ocurre cuando dos o más estudiantes comparten la misma suma total de créditos. El algoritmo de ordenación procedería entonces a ordenar las tuplas comparando los objetos estudiante entre sí. Esto no tiene ningún sentido y, en casos más complejos, podría conducir a resultados impredecibles, o incluso a errores. Si quiere estar seguro de evitar este problema, una solución sencilla es crear una tripleta en lugar de una dobleta, con la suma de créditos en primer lugar, la posición del objeto alumno en la lista de alumnos en segundo lugar, y el propio objeto alumno en tercer lugar. De esta forma, si la suma de créditos es la misma, las tuplas se ordenarán en función de la posición, que siempre será diferente, y por tanto suficiente para resolver la ordenación entre cualquier par de tuplas.

## zip
Según la documentación de Python, *`zip(*iterables)`* devuelve un iterador de tuplas, donde la i-ésima tupla contiene el i-ésimo elemento de cada una de las secuencias o iterables del argumento. El iterador se detiene cuando se agota el iterable de entrada más corto. Con un único argumento iterable, devuelve un iterador de 1-tuplas. Sin argumentos, devuelve un iterador vacío.

Veamos un ejemplo:

In [58]:
grades = [18, 35, 30, 24]
avgs = [23, 19, 32, 27]
print(f"{list(zip(avgs, grades)) = }")
print(f"Equivalente a usar zip(): {list(map(lambda *a: a, avgs, grades)) = }")

list(zip(avgs, grades)) = [(23, 18), (19, 35), (32, 30), (27, 24)]
Equivalente a usar zip(): list(map(lambda *a: a, avgs, grades)) = [(23, 18), (19, 35), (32, 30), (27, 24)]


En este ejercicio, hemos comprimido (zipping) la media (avgs) y la nota del último examen de cada estudiante (grades). Observe lo fácil que es reproducir *zip()* utilizando *map()*. Para visualizar los resultados, tenemos que utilizar *list()* al igual que lo hicimos con *map()*.

Un ejemplo sencillo del uso combinado de *map()* y *zip()* podría ser una forma de calcular el máximo por elemento entre secuencias; es decir, el máximo del primer elemento de cada secuencia, luego el máximo del segundo, y así sucesivamente:

In [63]:
a = [14, 65, 48, 32, 98]
b = [18, 98, 75, 45, 36]
c = [52, 11, 98, 65, 24]
maxs = map(lambda n: max(*n), zip(a, b, c))
list(maxs)

[52, 98, 98, 65, 98]

Observe lo fácil que es calcular los valores máximos de tres secuencias. *zip()* no es estrictamente necesario, por supuesto; podríamos utilizar *map()*. A veces es difícil, al mostrar un ejemplo sencillo, comprender por qué utilizar una técnica puede ser bueno o malo. Olvidamos que no siempre tenemos el control del código fuente; puede que tengamos que utilizar una biblioteca de terceros que no podemos cambiar como queremos. Disponer de distintas formas de trabajar con los datos es, por tanto, realmente útil.

## filter
Según la documentación de Python, *`filter(función, iterable)`* construye un iterador a partir de los elementos de iterable para los que la función devuelve *True*. iterable puede ser una secuencia, un contenedor que admita iteración o un iterador. Si la función es *None*, se asume la función identidad, es decir, se eliminan todos los elementos de iterable que sean *False*.

Veamos un ejemplo muy breve:

In [70]:
test = [2, 5, 8, 0, 0, 1, 0, 0, 0, 14, 12]
print(f"{list(filter(None, test)) = }")
print(f"Equivalente a la línea anterior: {list(filter(lambda x: x, test)) = }")
print(f"Guardar los items mayores a 7: {list(filter(lambda x: x > 7, test)) = }")

list(filter(None, test)) = [2, 5, 8, 1, 14, 12]
Equivalente a la línea anterior: list(filter(lambda x: x, test)) = [2, 5, 8, 1, 14, 12]
Guardar los items mayores a 7: list(filter(lambda x: x > 7, test)) = [8, 14, 12]


Armados con *map()*, *zip()*, y *filter()* (y varias otras funciones de la librería estándar de Python) podemos manipular secuencias muy efectivamente. Pero estas funciones no son la única manera de hacerlo. Echemos un vistazo a una de las características más agradables de Python: las comprensiones.

# Comprensiones
Una *comprensión* es una notación concisa para realizar alguna operación en cada elemento de una colección de objetos, y/o seleccionar un subconjunto de elementos que satisfagan alguna condición. Están tomadas del lenguaje de programación funcional Haskell y, junto con los iteradores y generadores, contribuyen a dar a Python un sabor funcional.

Python ofrece diferentes tipos de comprensiones: lista, diccionario y set. En este caso, nos centraremos en las comprensiones de lista; una vez que las entendamos, los otros tipos serán bastante fáciles de comprender.

Empecemos con un ejemplo muy sencillo. Queremos calcular una lista con los cuadrados de los 10 primeros números naturales. Tres diferentes formas para realizar ello son:

In [80]:
# usando bucle for
squares_first = []
for n in range(6):
    squares_first.append(n * n)
print(f"El primer método con el uso del bucle for da como resultado: {squares_first = }")

# usando la función map
squares_second = list(map(lambda n: n * n, range(6)))
print(f"El segundo método con el uso de la función map da como resultado: {squares_second = }")

# usando comprensión de lista
squares_third = [n * n for n in range(6)]
print(f"El tercer método con el uso de comprensiones da como resultado: {squares_third = }")

El primer método con el uso del bucle for da como resultado: squares_first = [0, 1, 4, 9, 16, 25]
El segundo método con el uso de la función map da como resultado: squares_second = [0, 1, 4, 9, 16, 25]
El tercer método con el uso de comprensiones da como resultado: squares_third = [0, 1, 4, 9, 16, 25]


Así de sencillo, usando compresiones podemos escribir el código de manera más elegante y sencilla, colocando un bucle *for* entre corchetes. Ahora vamos a filtrar los cuadrados impares; para ello, vamos a calcularlo de tres maneras:

In [84]:
# usando bucle for y el condicional if
sq1 = []
for n in range(6):
    if not n % 2:
        sq1.append(n * n)
print(f"Usando bucle for y el condicional if: {sq1 = }")

# usando la función map y filter
sq2 = list(map(lambda n: n * n, filter(lambda n: not n % 2, range(6))))
print(f"Usando la función map y filter: {sq2 = }")

# usando comprensión de lista
sq3 = [n * n for n in range(6) if not n % 2]
print(f"Usando comprensión de lista: {sq3 = }")
print(f"Los tres resultados son iguales: {sq1 == sq2 == sq3}")

Usando bucle for y el condicional if: sq1 = [0, 4, 16]
Usando la función map y filter: sq2 = [0, 4, 16]
Usando comprensión de lista: sq3 = [0, 4, 16]
Los tres resultados son iguales: True


Según la documentación de Python, una comprensión de lista consiste en corchetes que contienen una expresión seguida de una cláusula *for*, y luego cero o más cláusulas *for* o *if*. El resultado será una nueva lista resultante de evaluar la expresión en el contexto de las cláusulas *for* e *if* que le siguen.

## Comprensiones anidadas
Veamos un ejemplo de bucles anidados. Es muy común cuando se trata de algoritmos tener que iterar sobre una secuencia utilizando dos marcadores de posición. El primero recorre toda la secuencia, de izquierda a derecha. El segundo también lo hace, pero empieza desde el primero, en lugar de 0. El concepto es el de probar todos los pares sin duplicación. Veamos el equivalente clásico del bucle for:

In [3]:
items = 'ABCDEFG'
pairs = []
for a in range(len(items)):
    for b in range(a, len(items)):
        pairs.append((items[a], items[b]))
print(f"{pairs = }")

pairs = [('A', 'A'), ('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('A', 'F'), ('A', 'G'), ('B', 'B'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('B', 'F'), ('B', 'G'), ('C', 'C'), ('C', 'D'), ('C', 'E'), ('C', 'F'), ('C', 'G'), ('D', 'D'), ('D', 'E'), ('D', 'F'), ('D', 'G'), ('E', 'E'), ('E', 'F'), ('E', 'G'), ('F', 'F'), ('F', 'G'), ('G', 'G')]


Todas las tuplas con la misma letra son aquellas en las que *b* está en la misma posición que *a*. Ahora, veamos cómo podemos traducir esto a una comprensión de lista:

In [11]:
items = 'ABCDEFG'
pairs = [(items[a], items[b]) 
    for a in range(len(items)) for b in range(a, len(items))]
print(f"{pairs = }")

pairs = [('A', 'A'), ('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('A', 'F'), ('A', 'G'), ('B', 'B'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('B', 'F'), ('B', 'G'), ('C', 'C'), ('C', 'D'), ('C', 'E'), ('C', 'F'), ('C', 'G'), ('D', 'D'), ('D', 'E'), ('D', 'F'), ('D', 'G'), ('E', 'E'), ('E', 'F'), ('E', 'G'), ('F', 'F'), ('F', 'G'), ('G', 'G')]


Esta versión sólo tiene dos líneas y consigue el mismo resultado. Observe que en este caso particular, como el bucle for sobre b depende de a, debe seguir al bucle for sobre a en la comprensión. Si los intercambias, obtendrás un error de nombre.

Otra forma de conseguir el mismo resultado es utilizar la función *combinations_with_replacement()* del módulo *itertools*.

## Filtrar una comprensión
También podemos aplicar filtros a una comprensión. Primero usaremos *filter()*, y encontremos todos los triples pitagóricos ($a^2 + b^2 = c^2$) cuyos lados cortos sean números menores que 50. Obviamente, no queremos probar una combinación dos veces, y por ello utilizaremos un truco similar al que vimos en el ejemplo anterior: 

In [22]:
from math import sqrt
mx = 50
# crea todos los pares posibles
triples = [(a, b, sqrt(a**2 + b**2))
    for a in range(1, mx) for b in range(a, mx)]

# filtra todos los triples no pitagóricos
triples = list(
    filter(lambda triple: triple[2].is_integer(), triples))
print(triples)

[(3, 4, 5.0), (5, 12, 13.0), (6, 8, 10.0), (7, 24, 25.0), (8, 15, 17.0), (9, 12, 15.0), (9, 40, 41.0), (10, 24, 26.0), (12, 16, 20.0), (12, 35, 37.0), (14, 48, 50.0), (15, 20, 25.0), (15, 36, 39.0), (16, 30, 34.0), (18, 24, 30.0), (20, 21, 29.0), (20, 48, 52.0), (21, 28, 35.0), (24, 32, 40.0), (24, 45, 51.0), (27, 36, 45.0), (28, 45, 53.0), (30, 40, 50.0), (33, 44, 55.0), (36, 48, 60.0), (40, 42, 58.0)]


En el código anterior, hemos generado una lista de tres tuplas, *triples*. Cada tupla contiene dos números enteros (los catetos) y la hipotenusa del triángulo pitagórico cuyos catetos son los dos primeros números de la tupla. Por ejemplo, cuando a es 3 y b es 4, la tupla será (3, 4, 5,0), y cuando a es 5 y b es 7, la tupla será (5, 7, 8,602325267042627).

Después de generar todas las tripletas, tenemos que filtrar todas aquellas en las que la hipotenusa no sea un número entero. Para ello, filtramos basándonos en que *float_number.is_integer()* sea *True*. Esto significa que, de las dos tuplas de ejemplo que acabamos de mostrar, la que tiene la hipotenusa 5.0 se conservará, mientras que la que tiene la hipotenusa 8.602325267042627 se descartará. Esto está bien, pero no nos gusta el hecho de que la tripleta tenga dos números enteros y un flotante: se supone que todos son enteros. Usemos map() para arreglar esto:

In [34]:
from math import sqrt
mx = 50
# crea todos los pares posibles
triples = [(a, b, sqrt(a**2 + b**2))
    for a in range(1, mx) for b in range(a, mx)]
triples = filter(lambda triple: triple[2].is_integer(), triples)

# esto hará que el tercer número de las tuplas sea entero
triples = list(
    map(lambda triple: triple[:2] + (int(triple[2]), ), triples))
print(triples)

[(3, 4, 5), (5, 12, 13), (6, 8, 10), (7, 24, 25), (8, 15, 17), (9, 12, 15), (9, 40, 41), (10, 24, 26), (12, 16, 20), (12, 35, 37), (14, 48, 50), (15, 20, 25), (15, 36, 39), (16, 30, 34), (18, 24, 30), (20, 21, 29), (20, 48, 52), (21, 28, 35), (24, 32, 40), (24, 45, 51), (27, 36, 45), (28, 45, 53), (30, 40, 50), (33, 44, 55), (36, 48, 60), (40, 42, 58)]


Fíjate en el paso que hemos añadido. Tomamos cada elemento en triples y lo cortamos, tomando sólo los dos primeros elementos en él. Luego, concatenamos la rebanada con una tupla, en la que ponemos la versión entera de ese número flotante que no nos gustó. Otra forma cómo hacer todo esto con una comprensión de lista es la siguiente:

In [35]:
from math import sqrt
mx = 50
triples = [(a, b, sqrt(a**2 + b**2))
    for a in range(1, mx) for b in range(a, mx)]
# Aquí se combina filter y map en una comprensión de lista más limpia
triples = [(a, b, int(c)) for a, b, c in triples if c.is_integer()]
print(triples)

[(3, 4, 5), (5, 12, 13), (6, 8, 10), (7, 24, 25), (8, 15, 17), (9, 12, 15), (9, 40, 41), (10, 24, 26), (12, 16, 20), (12, 35, 37), (14, 48, 50), (15, 20, 25), (15, 36, 39), (16, 30, 34), (18, 24, 30), (20, 21, 29), (20, 48, 52), (21, 28, 35), (24, 32, 40), (24, 45, 51), (27, 36, 45), (28, 45, 53), (30, 40, 50), (33, 44, 55), (36, 48, 60), (40, 42, 58)]


Así está mucho mejor. Es limpio, legible y más corto. Sin embargo, no es tan elegante como podría haber sido. Seguimos desperdiciando memoria al construir una lista con muchos *triples* que acabamos descartando. Podemos arreglarlo combinando las dos comprensiones en una:

In [36]:
from math import sqrt
mx = 50
triples = [(a, b, int(c))
    for a in range(1, mx) for b in range(a, mx)
    if (c := sqrt(a**2 + b**2)).is_integer()]
print(triples)

[(3, 4, 5), (5, 12, 13), (6, 8, 10), (7, 24, 25), (8, 15, 17), (9, 12, 15), (9, 40, 41), (10, 24, 26), (12, 16, 20), (12, 35, 37), (14, 48, 50), (15, 20, 25), (15, 36, 39), (16, 30, 34), (18, 24, 30), (20, 21, 29), (20, 48, 52), (21, 28, 35), (24, 32, 40), (24, 45, 51), (27, 36, 45), (28, 45, 53), (30, 40, 50), (33, 44, 55), (36, 48, 60), (40, 42, 58)]


Eso sí que es elegante. Al generar las tripletas y filtrarlas en la misma comprensión de lista, evitamos mantener en memoria cualquier tripleta que no pase la prueba. Observa que hemos utilizado una expresión de asignación para evitar tener que calcular el valor de $sqrt(a^2 + b^2)$ dos veces.

## Comprensiones de diccionario
Las comprensiones de diccionario funcionan exactamente igual que las comprensiones de lista, pero para construir diccionarios, sólo hay una ligera diferencia en la sintaxis. El siguiente ejemplo será suficiente para explicar todo lo que necesita saber:

In [43]:
from codecs import ascii_decode
from string import ascii_lowercase
ascii_lowercase
lettermap = {c: k for k, c in enumerate(ascii_lowercase, 1)}
print(lettermap)

<built-in function ascii_decode>


En el código anterior, estamos enumerando la secuencia de todas las letras ASCII minúsculas (utilizando la función enumerar). A continuación, construimos un diccionario con los pares letra/número resultantes como claves y valores. Observe que la sintaxis es similar a la conocida sintaxis de diccionario. También hay otra forma de hacer lo mismo:

In [44]:
lettermap = dict((c, k) for k, c in enumerate(ascii_lowercase, 1))
print(lettermap)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


En este caso, estamos alimentando una expresión generadora al constructor dict. Tener en cuenta que los diccionarios no permiten claves duplicadas, como se muestra en el siguiente ejemplo:

In [45]:
word = 'Hello'
swaps = {c: c.swapcase() for c in word}
print(swaps)

{'H': 'h', 'e': 'E', 'l': 'L', 'o': 'O'}


Creamos un diccionario con las letras de la cadena 'Hello' como claves y las mismas letras, pero con las mayúsculas intercambiadas, como valores. Observa que sólo hay un par 'l': 'L'. El constructor no se queja; simplemente reasigna los duplicados al último valor. Aclaremos esto con otro ejemplo que asigna a cada tecla su posición en la cadena:

In [46]:
word = 'Hello'
positions = {c: k for k, c in enumerate(word)}
print(positions)

{'H': 0, 'e': 1, 'l': 3, 'o': 4}


Fíjate en el valor asociado a la letra 'l': 3. El par 'l': 2 no está ahí; ha sido anulado por 'l': 3.

## Comprensiones de set (conjuntos)
Las comprensiones de conjuntos son muy similares a las de listas y diccionarios. Veamos un ejemplo rápido:

In [49]:
word = 'Hello'
letters1 = {c for c in word}
letters2 = set(c for c in word)
print(f"{letters1 = }")
print(f"{letters1 == letters2 = }")

letters1 = {'H', 'l', 'e', 'o'}
letters1 == letters2 = True


Notar que para las comprensiones de conjuntos, al igual que para los diccionarios, no se permite la duplicación, por lo que el conjunto resultante sólo tiene cuatro letras. Observe también que las expresiones asignadas a letters1 y letters2 producen conjuntos equivalentes.

La sintaxis utilizada para crear letters11 es muy similar a la de una comprensión de diccionario. La única diferencia es que los diccionarios requieren claves y valores separados por dos puntos, mientras que los conjuntos no. Para letters2, hemos introducido una expresión generadora en el constructor set().

# Generadores
Los generadores son herramientas muy potentes. Se basan en el concepto de iteración y permiten patrones de codificación que combinan elegancia y eficiencia.

Los generadores son de dos tipos:
- **Funciones generadoras:** Son muy similares a las funciones regulares, pero en lugar de devolver resultados mediante sentencias *return*, utilizan *yield*, lo que les permite suspender y reanudar su estado entre cada llamada.
- **Expresiones generadoras:** Son muy similares a las comprensiones de listas que hemos visto anteriormente, pero en lugar de devolver una lista, devuelven un objeto que produce resultados uno a uno.

## Funciones generadoras
Las funciones generadoras se comportan como funciones normales en todos los aspectos, excepto en una diferencia: en lugar de recoger los resultados y devolverlos a la vez, se convierten automáticamente en iteradores que devuelven los resultados de uno en uno cuando se les llama *next*. Python convierte automáticamente las funciones generadoras en sus propios iteradores.

Todo esto es muy teórico, así que vamos a aclarar por qué un mecanismo así es tan potente, y luego veamos un ejemplo. Digamos que te pedimos que cuentes en voz alta de 1 a 1.000.000. Empiezas y, en algún momento, te pedimos que pares. Al cabo de un rato, le pedimos que reanude la cuenta. En ese momento, ¿cuál es la información mínima que necesitas para poder reanudar la cuenta correctamente? Bueno, necesitas recordar el último número al que llamaste. Si te hemos parado después del 31.415, seguirás con el 31.416, y así sucesivamente.

La cuestión es que no hace falta que recuerdes todos los números que dijiste antes del 31.415, ni que los tengas apuntados en algún sitio. Puede que no lo sepas, ¡pero ya te estás comportando como un generador!

Fíjate bien en el siguiente código:

In [1]:
# enfoque clásico de la función
def get_squares(n):
    return [x ** 2 for x in range(n)]
print(get_squares(10))

# enfoque generador (usamos yield y no return)
def get_squares_gen(n):
    for x in range(n):
        yield x ** 2
print(list(get_squares_gen(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


El resultado de las dos sentencias print será el mismo: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]. Pero hay una gran diferencia entre las dos funciones.

*get_squares()* es una función clásica que recoge todos los cuadrados de números en [0, n) en una lista, y la devuelve. En cambio, *get_squares_gen()* es un generador y se comporta de forma muy diferente. Cada vez que el intérprete llega a la línea *yield*, su ejecución se suspende. La única razón por la que esas sentencias *print* devuelven el mismo resultado es porque hemos alimentado *get_squares_gen()* al constructor *list()*, que agota completamente el generador pidiendo el siguiente elemento hasta que se produce un *StopIteration*. Veamos esto en detalle:

In [5]:
def get_squares_gen(n):
    for x in range(n):
        yield x ** 2

squares = get_squares_gen(4) # esto crea un objeto generador
print(squares)
print(next(squares)) # prints: 0
print(next(squares)) # prints: 1
print(next(squares)) # prints: 4
print(next(squares)) # prints: 9
try:
    print(next(squares))    # Esto lanzará una excepción StopIteration
except StopIteration as e:  # Corrección: especifica la excepción StopIteration y usa "as e"
    print(e)                # Muestra la excepción, que será StopIteration

<generator object get_squares_gen at 0x000002655C8C8860>
0
1
4
9



Cada vez que llamamos a *next* en el objeto generador, lo iniciamos (el primer *next*) o hacemos que se reanude desde el último punto de suspensión (cualquier otro *next*). La primera vez que llamamos next sobre él, obtenemos 0, que es el cuadrado de 0, luego 1, luego 4, luego 9, y como el bucle *for* se detiene después de eso (n es 4), el generador termina naturalmente. Una función clásica devolvería None, pero para cumplir con el protocolo de iteración, un generador lanzará una excepción *StopIteration*.

Vale la pena señalar que se puede utilizar la sentencia *return* en una función generadora. Causará una excepción *StopIteration* que se planteará, terminando efectivamente la iteración. Esto es extremadamente importante. Si una sentencia return hiciera que la función devolviera algo, se rompería el protocolo de iteración. La consistencia de Python evita esto, y nos permite una gran facilidad a la hora de codificar. Veamos un ejemplo rápido:

In [1]:
def geometric_progression(a, q):
    k = 0
    while True:
        result = a * q**k
        if result <= 100000:
            yield result
        else:
            return
        k += 1

for n in geometric_progression(2, 5):
    print(n)

2
10
50
250
1250
6250
31250


El código anterior produce todos los términos de la progresión geométrica, $a, aq, aq^2, aq^3, ....$ Cuando la progresión produce un término mayor que 100.000, el generador se detiene (con una sentencia return).

## Más allá del próximo
Los objetos generadores se basan en el protocolo de iteración. Tener en cuenta que cuando llamas a *next(generator)*, estás llamando al método `generator.__next__()`. Además, se debe recordar que un método es sólo una función que pertenece a un objeto, y los objetos en Python pueden tener métodos especiales. `__next__()` es sólo uno de ellos y su propósito es devolver el siguiente elemento de la iteración, o lanzar *StopIteration* cuando la iteración ha terminado y no hay más elementos que devolver.

Cuando escribimos una función generadora, Python la transforma automáticamente en un objeto muy similar a un iterador, y cuando llamamos a *next(generador)*, esa llamada se transforma en `generator.__next__()`. Volvamos al ejemplo anterior sobre la generación de cuadrados:

In [11]:
def get_squares_gen(n):
    for x in range(n):
        yield x ** 2
squares = get_squares_gen(3)
print(squares.__next__()) # prints: 0
print(squares.__next__()) # prints: 1
print(squares.__next__()) # prints: 4
try:
    print(squares.__next__())       # Esto lanzará una excepción StopIteration
except StopIteration as e:          # Corrección: especifica la excepción StopIteration y usa "as e"
    print(e)                        # Muestra la excepción, que será StopIteration

0
1
4



El resultado es exactamente el mismo que en el ejemplo anterior, sólo que esta vez en lugar de utilizar la llamada proxy *next(squares)*, estamos llamando directamente a `squares.__next__()`.

Los objetos generadores también tienen otros tres métodos que nos permiten controlar su comportamiento: `send()`, `throw()` y `close()`. El primero nos permite comunicar un valor de vuelta al objeto generador, mientras que el segundo y tercer método nos permiten lanzar una excepción dentro del generador y cerrarlo, respectivamente. Su uso es bastante avanzado y no vamos a cubrirlos aquí en detalle, pero queremos dedicar unas palabras a `send()`, con un ejemplo sencillo:

In [12]:
def counter(start=0):
    n = start
    while True:
        yield n
        n += 1
e = counter()
print(next(e)) # prints: 0
print(next(e)) # prints: 1
print(next(e)) # prints: 2

0
1
2


El iterador anterior crea un objeto generador que se ejecutará para siempre. Puedes seguir llamándolo y nunca se detendrá. Alternativamente, puedes ponerlo en un bucle for, por ejemplo, `for n in counter(): ...`, y también seguirá para siempre. Pero, ¿y si quieres detenerlo en algún momento? Una solución es utilizar una variable para controlar el bucle *while*, en algo como esto:

In [13]:
stop = False
def counter(start=0):
    n = start
    while not stop:
        yield n
        n += 1
c = counter()
print(next(c)) # prints: 0
print(next(c)) # prints: 1
stop = True
try:
    print(next(c))                  # Esto lanzará una excepción StopIteration porque stop es True
except StopIteration as e:          # Corrección: especifica la excepción StopIteration y usa "as e"
    print(e)                        # Muestra la excepción, que será StopIteration

0
1



Empezamos con *stop = False*, y hasta que lo cambiemos a *True*, el generador seguirá funcionando, como antes. Sin embargo, en el momento en que cambiemos *stop* a *True*, el bucle *while* saldrá, y la siguiente llamada a *next* lanzará una excepción *StopIteration*. Este truco funciona, pero no es muy elegante ya que se depende de una variable externa, y esto puede dar problemas.

Podemos mejorarlo utilizando *generator.send()*. Cuando llamamos a *generator.send()*, el valor que introducimos para enviar se pasa al generador, se reanuda la ejecución y podemos recuperarlo mediante la expresión *yield*. Veamos un ejemplo:

In [18]:
def counter(start=0):
    n = start
    while True:
        result = yield n            # A
        print(type(result), result) # B
        if result == 'Q':
            break
        n += 1
c = counter()
print(next(c))          # C
print(c.send('Wow!'))   # D
print(next(c))          # E
try:
    print(c.send('Q'))  # F
except StopIteration as e:
    print(e)


0
<class 'str'> Wow!
1
<class 'NoneType'> None
2
<class 'str'> Q



Vamos a repasar este código línea a línea entender lo que está pasando.

Comenzamos la ejecución del generador con una llamada a *next()* (#C). Dentro del generador, *n* se pone al mismo valor que *start*. Se entra en el bucle *while*, la ejecución se detiene (#A), y *n* (0) se devuelve a quien lo llamó. *0* se imprime en la consola.

A continuación, llamamos a *send()* (#D), la ejecución se reanuda, el resultado se establece en *'¡Wow!'* (todavía #A), y luego su tipo y valor se imprimen en la consola (#B). El resultado no es *'Q'*, por lo que *n* se incrementa en 1 y la ejecución vuelve a la condición *while*, que, siendo *True*, se evalúa a *True*. Otro ciclo de bucle comienza, la ejecución se detiene de nuevo (#A), y *n* (1) se devuelve a la persona que llama. *1* se imprime en la consola.

En este punto, llamamos a *next()* (#E), la ejecución se reanuda de nuevo (#A), y como no estamos enviando nada al generador explícitamente, la expresión *yield* *n* (#A) devuelve *None* (el comportamiento es exactamente el mismo que cuando llamamos a una función que no devuelve nada). *result* se establece por lo tanto a *None*, y su tipo y valor se imprimen de nuevo en la consola (#B). La ejecución continúa, el resultado no es *'Q'*, por lo que *n* se incrementa en 1, y comenzamos de nuevo otro bucle. La ejecución se detiene de nuevo (#A) y *n* (2) se devuelve al invocador. *2* se imprime en la consola.

Y ahora el gran final: volvemos a llamar a *send()* (#F), pero esta vez le pasamos *'Q'*, y así cuando se reanuda la ejecución, el resultado es *'Q'* (#A). Su tipo y valor se imprimen en la consola (#B), y finalmente la cláusula *if* se evalúa a *True* y el bucle while se detiene mediante la sentencia *break*. El generador termina naturalmente, lo que significa que se lanza una excepción *StopIteration*.

Esto no es nada sencillo de entender a primera vista. El uso de *send()* permite patrones interesantes, y vale la pena señalar que *send()* también se puede utilizar para iniciar la ejecución de un generador (siempre que lo llames con *None*).

## La expresión yield from
Otra construcción interesante es la expresión `yield from`. Esta expresión permite obtener valores de un *sub-iterador*. Su uso permite patrones bastante avanzados, así que veamos un ejemplo muy rápido de ello:


In [19]:
def print_squares(start, end):
    for n in range(start, end):
        yield n ** 2
for n in print_squares(2, 5):
    print(n)

4
9
16


El código anterior imprime los números 4, 9 y 16 en la consola (en líneas separadas). El bucle *for* fuera de la función obtiene un iterador de *print_squares(2, 5)* y llama a *next()* sobre él hasta que termina la iteración. Cada vez que se llama al generador, la ejecución se suspende (y más tarde se reanuda) en *yield* $n ** 2$, que devuelve el cuadrado del n actual. Veamos cómo podemos transformar este código, beneficiándonos de la expresión *yield from*:

In [20]:
def print_squares(start, end):
    yield from (n ** 2 for n in range(start, end))
for n in print_squares(2, 5):
    print(n)

4
9
16


Este código produce el mismo resultado, pero como puedes ver, *yield from* en realidad está ejecutando un sub-iterador, $(n ** 2 ...)$. La expresión *yield from* devuelve al invocador cada valor que el sub-iterador está produciendo. Es más corto y se lee mejor.

## Expresiones del generador
Hablemos ahora de la otra técnica para generar valores de uno en uno. La sintaxis es exactamente la misma que la de las comprensiones de lista, sólo que, en lugar de envolver la comprensión con corchetes, la envuelves con corchetes redondos. Esto se denomina expresión generadora. 

En general, las expresiones generadoras se comportan como las comprensiones de lista equivalentes, pero hay una cosa muy importante que recordar: los generadores sólo permiten una iteración, después se agotarán.

Revisemos el siguiente ejemplo:

In [3]:
cubes = [k**3 for k in range(10)] # regular list
print(f"{cubes = }")
print(f"{type(cubes) = }")

cubes_gen = (k**3 for k in range(10)) # create as generator
print(f"{type(cubes_gen) = }")
print(f"{list(cubes_gen) = }") # this will exhaust the generator
print(f"{list(cubes_gen) = }") # nothing more to give

cubes = [0, 1, 8, 27, 64, 125, 216, 343, 512, 729]
type(cubes) = <class 'list'>
type(cubes_gen) = <class 'generator'>
list(cubes_gen) = [0, 1, 8, 27, 64, 125, 216, 343, 512, 729]
list(cubes_gen) = []


Si nos percatamos en la línea en la que se crea la expresión generadora y se le asigna el nombre *cubes_gen*; puedes ver que es un objeto generador. Para ver sus elementos, podemos utilizar un bucle *for*, un conjunto manual de llamadas a *next*, o simplemente alimentarlo a un constructor *list()*, que es lo que hemos hecho.

Observa cómo, una vez agotado el generador, no hay forma de volver a recuperar los mismos elementos de él. Tenemos que recrearlo si queremos volver a utilizarlo desde cero.

En los siguientes ejemplos, vamos a ver cómo reproducir *map()* y *filter()* utilizando expresiones del generador. Comenzamos con *map()*:

In [4]:
def adder(*n):
    return sum(n)
s1 = sum(map(adder, range(100), range(1, 101)))
print(f"{s1 = }")
s2 = sum(adder(*n) for n in zip(range(100), range(1, 101)))
print(f"{s2 = }")
print(f"{s1 == s2 = }")

s1 = 10000
s2 = 10000
s1 == s2 = True


En el ejemplo anterior, *s1* y *s2* son exactamente iguales: son la suma de *adder(0, 1)*, *adder(1, 2)*, *adder(2, 3)*, y así sucesivamente, lo que se traduce en sum(1, 3, 5, ...). La sintaxis es diferente, aunque la expresión del generador nos parece mucho más legible. Ahora, para *filter()*:

In [14]:
cubes = [x**3 for x in range(10)]
print(f"{cubes = }")
odd_cubes1 = filter(lambda cube: cube % 2, cubes)
print(f"{list(odd_cubes1) = }")
odd_cubes2 = (cube for cube in cubes if cube % 2)
print(f"{list(odd_cubes2) = }")
print(f"{list(odd_cubes1) == list(odd_cubes2) = }")

cubes = [0, 1, 8, 27, 64, 125, 216, 343, 512, 729]
list(odd_cubes1) = [1, 27, 125, 343, 729]
list(odd_cubes2) = [1, 27, 125, 343, 729]
list(odd_cubes1) == list(odd_cubes2) = True


En este ejemplo, *odd_cubes1* y *odd_cubes2* son lo mismo: generan una secuencia de cubos impares. Una vez más, preferimos la sintaxis del generador. Esto debería ser evidente cuando las cosas se pongan un poco más complicadas:

In [17]:
N = 20
cubes1 = map(
    lambda n: (n, n**3),
    filter(lambda n: n % 3 == 0 or n % 5 == 0, range(N)))
print(f"{list(cubes1) = }")
cubes2 = (
    (n, n**3) for n in range(N) if n % 3 == 0 or n % 5 == 0)
print(f"{list(cubes2) = }")
print(f"{list(cubes1) == list(cubes2) = }")

list(cubes1) = [(0, 0), (3, 27), (5, 125), (6, 216), (9, 729), (10, 1000), (12, 1728), (15, 3375), (18, 5832)]
list(cubes2) = [(0, 0), (3, 27), (5, 125), (6, 216), (9, 729), (10, 1000), (12, 1728), (15, 3375), (18, 5832)]
list(cubes1) == list(cubes2) = True


El código anterior crea dos generadores, *cubes1* y *cubes2*. Son exactamente iguales y devuelven dos tuplas *(n, `n^3`)* cuando n es múltiplo de 3 o de 5.

Continuando con los ejercicios, veremos la diferencia de las siguientes lineas de código:

In [19]:
s1 = sum([n**2 for n in range(10**6)])
print(f"{s1 = }")
s2 = sum((n**2 for n in range(10**6)))
print(f"{s2 = }")
s3 = sum(n**2 for n in range(10**6))
print(f"{s3 = }")

s1 = 333332833333500000
s2 = 333332833333500000
s3 = 333332833333500000


De manera estricta, todas producen la misma suma. Las expresiones para obtener *s2* y *s3* son exactamente iguales porque los paréntesis en *s2* son redundantes. Ambas son expresiones generadoras dentro de la función *sum()*. Sin embargo, la expresión para obtener *s1* es diferente. Dentro de *sum()*, encontramos una comprensión de lista. Esto significa que para calcular *s1*, la función *sum()* tiene que llamar a *next()* en una lista un millón de veces.

¿Ves dónde estamos perdiendo tiempo y memoria? Antes de que *sum()* pueda empezar a llamar a *next()* en esa lista, la lista tiene que haber sido creada, lo que es una pérdida de tiempo y espacio. Es mucho mejor que *sum()* llame a *next()* sobre una simple expresión generadora. No hay necesidad de tener todos los números de _range(10**6)_ almacenados en una lista.
Por lo tanto, tenga cuidado con los paréntesis extra cuando escriba sus expresiones. A veces es fácil pasar por alto estos detalles que hacen que nuestro código sea muy diferente.

Veamos el siguiente ejemplo:

In [1]:
# example1 = sum([n**2 for n in range(10**9)])
example2 = sum(n**2 for n in range(10**9))
print(f"{example2 = }")

example2 = 333333332833333333500000000


La diferencia entre las dos líneas, es que en la primera (la que esta comentada) hay que hacer una lista con los cuadrados de los primeros mil millones de números antes de poder sumarlos. Esa lista es enorme, así que necesitamos mucha memoria: y en el caso de que nos quedemos sin memoria, Python mata el proceso por nosotros.

Pero cuando quitamos los corchetes, ya no tenemos una lista. La función *sum* recibe 0, 1, 4, 9, y así hasta el último, y los suma. Sin problemas.

# Algunas consideraciones sobre el rendimiento
Hemos visto que tenemos muchas formas diferentes de conseguir el mismo resultado. Podemos utilizar cualquier combinación de `map()`, `zip()`, y `filter()`, u optar por una `comprensión` o un `generador`. Incluso podemos decidirnos por bucles `for`; cuando la lógica a aplicar a cada parámetro en ejecución no es sencilla, éstos pueden ser la mejor opción.

Además de la legibilidad, hablemos del rendimiento. Cuando se trata de rendimiento, normalmente hay dos factores que juegan un papel importante: el `espacio` y el `tiempo`. 

El `espacio` se refiere al tamaño de la memoria que va a ocupar una estructura de datos. La mejor manera de elegir es preguntarte si realmente necesitas una lista (o tupla), o si en su lugar funcionaría una simple función generadora. Si la respuesta es afirmativa, opta por el generador, ya que te ahorrará mucho espacio. Lo mismo ocurre con las funciones: si no necesitas que devuelvan una lista o tupla, también puedes transformarlas en funciones generadoras.

A veces, tendrás que usar listas (o tuplas); por ejemplo, hay algoritmos que escanean secuencias usando múltiples punteros, o quizás recorren la secuencia más de una vez. Una función generadora (o expresión) sólo puede iterarse sobre ella una vez y luego se agota, por lo que en estas situaciones no sería la elección correcta.

El `tiempo` es un poco más complicado que el espacio porque depende de más variables, y por tanto no es posible afirmar que *X* es más rápido que *Y* con absoluta certeza para todos los casos. Sin embargo, basándonos en las pruebas realizadas hoy en Python, podemos decir que, en promedio, `map()` muestra un rendimiento similar al de las `comprensiones` y las `expresiones generadoras`, mientras que los bucles `for` son sistemáticamente más lentos. 

Para poder apreciar plenamente el razonamiento que hay detrás de estas afirmaciones, necesitamos entender cómo funciona Python. En palabras sencillas, `map()` y las `comprensiones` se ejecutan a la velocidad del lenguaje C dentro del intérprete, mientras que un bucle `for` de Python se ejecuta como bytecode de Python dentro de la Máquina Virtual de Python, que suele ser mucho más lenta.

Escribiremos un pequeño trozo de código que recoja los resultados de `divmod(a, b)` para un determinado conjunto de pares de enteros, `(a, b)`. Utilizaremos la función `time()` del módulo `time` para calcular el tiempo transcurrido de las operaciones que vamos a realizar:

In [3]:
from time import time
mx = 5000
t = time() # start time for the for loop
floop = []
for a in range(1, mx):
    for b in range(a, mx):
        floop.append(divmod(a, b))
print('for loop: {:.4f} s'.format(time() - t)) # elapsed time

t = time() # start time for the list comprehension
compr = [
    divmod(a, b) for a in range(1, mx) for b in range(a, mx)]
print('list comprehension: {:.4f} s'.format(time() - t))

t = time() # start time for the generator expression
gener = list(
    divmod(a, b) for a in range(1, mx) for b in range(a, mx))
print('generator expression: {:.4f} s'.format(time() - t))

for loop: 4.7300 s
list comprehension: 3.9027 s
generator expression: 3.6191 s


La *comprensión de la lista* se ejecuta en un tiempo menor que la del bucle *for* y la expresión del generador. La diferencia de tiempo entre la comprensión de la lista y la expresión del generador apenas es significativa.

Un resultado interesante es observar que, dentro del cuerpo del bucle *for*, estamos añadiendo datos a una lista. Esto implica que Python hace el trabajo, entre vestidores, de redimensionarla de vez en cuando, asignando espacio para los elementos que se van a añadir.

Veamos un ejemplo similar que compara un bucle for y una llamada a *map()*:

In [4]:
mx = 2 * 10 ** 7
t = time()
absloop = []
for n in range(mx):
    absloop.append(abs(n))
print('for loop: {:.4f} s'.format(time() - t))

t = time()
abslistcompr = [abs(n) for n in range(mx)]
print('list comprehension: {:.4f} s'.format(time() - t))

t = time()
absmap = list(map(abs, range(mx)))
print('map: {:.4f} s'.format(time() - t))

t = time()
abslistgen = list((abs(n) for n in range(mx)))
print('generator expression: {:.4f} s'.format(time() - t))

for loop: 3.6240 s
list comprehension: 2.5155 s
map: 1.7900 s
generator expression: 2.8756 s


Este código es conceptualmente muy similar al ejemplo anterior. Lo único que ha cambiado es que estamos aplicando la función *abs()* en lugar de *divmod()*, y tenemos un solo bucle en lugar de dos anidados.

En este caso, *map* resulta ser el más veloz de todos. No obstante, hay que tener cuidado con estos resultados, ya que pueden variar en función de varios factores, como el sistema operativo y la versión de Python. Pero en general, creemos que es seguro decir que estos resultados son lo suficientemente buenos para tener una idea cuando se trata de codificar para el rendimiento.

Aparte de las pequeñas diferencias caso por caso, está bastante claro que la opción del bucle *for* es la más lenta, así que veamos por qué se sigue utilizando.

# No abuses de las comprensiones y los generadores
Ya hemos visto lo potentes que pueden ser las comprensiones y las expresiones generadoras. Pero a medida que tratamos con ellas, su complejidad crece exponencialmente. Cuanto más intentas hacer dentro de una sola comprensión o una expresión generadora, más difícil se hace leerla, entenderla y, por tanto, mantenerla o cambiarla.

Si consultamos el Zen de Python, hay unas cuantas líneas que creemos que merece la pena tener en cuenta cuando tratamos con código optimizado:

In [9]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


Las comprensiones y expresiones generadoras son más implícitas que explícitas, pueden ser bastante difíciles de leer y entender, y pueden ser difíciles de explicar. A veces, hay que desmenuzarlas utilizando la técnica de dentro-fuera para entender lo que está pasando.

Por poner un ejemplo, hablemos un poco más de los triples pitagóricos ($a^2 + b^2 = c^2$). Una forma de obtener una lista de triples pitagóricos de manera eficiente es generarlos directamente. Hay muchas fórmulas diferentes que se pueden utilizar para hacerlo; aquí utilizaremos la fórmula euclídea. Esta fórmula dice que cualquier triple (a, b, c), donde $a = m^2 - n^2$, $b = 2mn$ y $c = m^2 + n^2$, con *m* y *n* enteros positivos tales que $m > n$, es un triple pitagórico. Por ejemplo, cuando *m = 2* y *n = 1*, encontramos el triple más pequeño: *(3, 4, 5)*.

Sin embargo, hay una trampa: consideremos el triple *(6, 8, 10)*, que es como *(3, 4, 5)*, sólo que todos los números se multiplican por 2. Este triple es definitivamente pitagórico, ya que $6^2 + 8^2 = 102$, pero podemos derivarlo de (3, 4, 5) simplemente multiplicando cada uno de sus elementos por 2. En general, podemos escribir todos los triples como *(3k, 4k, 5k)*, siendo *k* un número entero positivo mayor que 1.

Un triple que no puede obtenerse multiplicando los elementos de otro por algún factor, k, se llama primitivo. Otra forma de expresarlo es: si los tres elementos de un triple son coprimos, entonces el triple es primitivo. Dos números son coprimos cuando no comparten ningún factor primo entre sus divisores, es decir, cuando su máximo común divisor (MCD) es 1. Por ejemplo, 3 y 5 son coprimos, mientras que 3 y 6 no lo son porque ambos son divisibles por 3.

Así, la fórmula euclidiana nos dice que si $m$ y $n$ son coprimos, y $m - n$ es impar, el triple que generan es primitivo. En el siguiente ejemplo, vamos a escribir una expresión generadora para calcular todos los triples pitagóricos primitivos cuya hipotenusa, *c*, es menor o igual que algún número entero, *N*. Esto significa que queremos todos los triples para los que $m^2 + n^2 ≤ N$. Cuando *n* es 1, la fórmula se parece a esto: $m^2 ≤ N - 1$, lo que significa que podemos aproximar el cálculo con un límite superior de $m ≤ N^1/2$.

Recapitulemos: $m$ debe ser mayor que $n$, también deben ser coprimos, y su diferencia $m - n$ debe ser impar. Además, para evitar cálculos inútiles, pondremos el límite superior de $m$ en $floor(sqrt(N)) + 1$.

La función $floor$ para un número real, *x*, da el máximo número entero, *n*, tal que $n < x$, por ejemplo, $floor(3.8) = 3$, $floor(13.1) = 13$. Tomar $floor(sqrt(N)) + 1$ significa tomar la parte entera de la raíz cuadrada de *N* y añadir un margen mínimo para asegurarnos de que no se nos escapa ningún número.

Pongamos todo esto en código, paso a paso. Empezamos escribiendo una sencilla función $gcd()$ que utiliza el **algoritmo de Euclides**:

In [12]:
def gcd(a, b):
    """Calculate the Greatest Common Divisor of (a, b). """
    while b != 0:
        a, b = b, a % b
    return a

El siguiente paso es utilizar los conocimientos que hemos reunido antes para generar una lista de triples pitagóricos primitivos:

In [14]:
N = 50
triples = sorted(                                   # 1
    ((a, b, c) for a, b, c in (                     # 2
        ((m**2 - n**2), (2 * m * n), (m**2 + n**2)) # 3
        for m in range(1, int(N**.5) + 1)           # 4
        for n in range(1, m)                        # 5
        if (m - n) % 2 and gcd(m, n) == 1           # 6
    ) if c <= N), key=sum                           # 7
)

[(3, 4, 5),
 (5, 12, 13),
 (15, 8, 17),
 (7, 24, 25),
 (21, 20, 29),
 (35, 12, 37),
 (9, 40, 41)]

No es fácil de leer el código anterior, así que vamos a ir a través de él línea por línea. En #3, empezamos una expresión generadora que crea triples. Puedes ver en #4 y #5 que estamos haciendo un bucle sobre *m* en $[1, M]$, siendo *M* la parte entera de $sqrt(N)$, más 1. Por otro lado, *n* hace un bucle dentro de $[1, m)$, para respetar la regla $m > n$. Vale la pena observar cómo hemos calculado sqrt(N), es decir, N**.5, que es otra forma de hacerlo.

En #6, puedes ver las condiciones de filtrado para que las tripletas sean primitivas: $(m - n) % 2$ se evalúa como $True$ cuando $(m - n)$ es impar, y $gcd(m, n) == 1$ significa que *m* y *n* son coprimos. Con esto, sabemos que los triples serán primitivos. Esto se ocupa de la expresión generadora más interna. La más externa empieza en #2 y termina en #7. Tomamos las triplas $(a, b, c)$ en (generador interno) tales que $c <= N$.

Finalmente, en #1, aplicamos la ordenación para presentar la lista en orden. En #7, después de cerrar la expresión del generador más externo, puedes ver que especificamos que la clave de ordenación sea la suma $a + b + c$. 

Ahora vamos a reescribir este código en algo más legible:

In [ ]:
def gen_triples(N):
    for m in range(1, int(N**.5) + 1):          # 1
        for n in range(1, m):                   # 2
            if (m - n) % 2 and gcd(m, n) == 1:  # 3
                c = m**2 + n**2                 # 4
                if c <= N:                      # 5
                    a = m**2 - n**2             # 6
                    b = 2 * m * n               # 7
                    yield (a, b, c)             # 8
sorted(gen_triples(50), key=sum)                # 9

Esto es mucho mejor de entender. 

Empezamos el bucle en #1 y #2, exactamente de la misma manera que en el ejemplo anterior. En la línea #3, tenemos el filtrado para las tripletas primitivas. En la línea 4, nos desviamos un poco de lo que hacíamos antes: calculamos *c*, y en la línea 5, filtramos si *c* es menor o igual que *N*. Sólo cuando *c* cumple esa condición, calculamos *a* y *b*, y obtenemos la tupla resultante. En la última línea, aplicamos la ordenación con la misma clave que estábamos utilizando en el ejemplo de la expresión generadora.

La moraleja de la historia es: intenta usar comprensiones y expresiones generadoras tanto como puedas, pero si el código empieza a ser complicado de modificar o leer, puede que quieras refactorizarlo en algo más legible.

# Localización de nombres
Ahora que estamos familiarizados con todos los tipos de comprensiones y expresiones generadoras, hablemos de la localización de nombres dentro de ellas. Python 3 localiza las variables de bucle en las cuatro formas de comprensión: lista, diccionario, conjunto y expresiones generadoras. Este comportamiento es, por tanto, diferente al del bucle *for*. Veamos algunos ejemplos sencillos para mostrar todos los casos:

In [15]:
A = 100
ex1 = [A for A in range(5)]
print(A) # prints: 100
ex2 = list(A for A in range(5))
print(A) # prints: 100
ex3 = {A: 2 * A for A in range(5)}
print(A) # prints: 100
ex4 = {A for A in range(5)}
print(A) # prints: 100
s = 0
for A in range(5):
    s += A
print(A) # prints: 4

100
100
100
100
4


En el código anterior, declaramos un nombre global, $A = 100$, y a continuación ejercitamos comprensiones de lista, diccionario y conjunto y una expresión generadora. Ninguna de ellas altera el nombre global, $A$. Por el contrario, puedes ver al final que el bucle *for* lo modifica. La última sentencia $print$ imprime 4.

Veamos qué pasaría si $B$ no estuviera ahí:

In [19]:
try:
    ex1 = [B for B in range(5)]
    print(B)            # Intento de acceso fuera del ámbito de la comprensión de lista
except NameError as e:  # Captura de la excepción correcta y usando 'NameError'
    print(f"{e = }")    # Imprime la excepción capturada

e = NameError("name 'B' is not defined")


El código anterior funcionaría del mismo modo con cualquier otro tipo de comprensión o con una expresión generadora. Después de ejecutar la primera línea, $B$ no está definido en el espacio de nombres global. Una vez más, el bucle *for* se comporta de forma diferente:

In [3]:
r = 0
for C in range(5):
    r += C
print(C)            # prints: 4
# print(globals())

4


El código anterior muestra que después de un bucle *for*, si la variable de bucle no estaba definida antes, podemos encontrarla en el marco global. Para asegurarnos de ello, echemos un vistazo llamando a la función incorporada $globals()$, donde podemos detectar 'C': 4.

# Comportamiento de la generación en los módulos integrados
El comportamiento tipo generador es bastante común entre los tipos y funciones incorporados. Esta es una diferencia importante entre Python 2 y Python 3. En Python 2, funciones como `map()`, `zip()` y `filter()` devolvían listas en lugar de objetos iterables. La idea detrás de este cambio es que si necesitas hacer una lista de esos resultados, siempre puedes envolver la llamada en una clase `list()`, y listo. Por otro lado, si sólo necesitas iterar y quieres mantener el impacto en memoria lo más ligero posible, puedes usar esas funciones de forma segura. Otro ejemplo notable es la función `range()`. En Python 2 devolvía una lista, y había otra función llamada `xrange()` que se comportaba como la función `range()` en Python 3.

La idea de funciones y métodos que devuelven objetos iterables está bastante extendida. Puedes encontrarla en la función `open()`, que se usa para operar con objetos archivo, pero también en `enumerate()`, en los métodos `keys()`, `values()` e `items()` del diccionario, y en varios otros lugares.

El objetivo de Python es intentar reducir la huella de memoria evitando malgastar espacio siempre que sea posible, especialmente en aquellas funciones y métodos que se utilizan mucho en la mayoría de las situaciones.

# Un último ejemplo
Recapitulando todo lo que hemos visto, vamos a escribir una función que devuelva los términos de la secuencia 0 1 1 2 3 5 8 13 21 ..., hasta cierto límite, $N$.

Es decir, vamos a realizar la sucesión de Fibonacci, que se define como $F(0) = 0$, $F(1) = 1$ y, para cualquier $n > 1$, $F(n) = F(n-1) + F(n-2)$. Esta secuencia es excelente para comprobar los conocimientos sobre recursividad, técnicas de memoización (técnica de programación en la cual se reduce el tiempo de ejecución de una función a cambio de ampliar el coste del espacio en memoria) y otros detalles técnicos.

Empecemos con una versión rudimentaria, y luego vamos a ir mejorandola:

In [4]:
def fibonacci(N):
    """Return all fibonacci numbers up to N. """
    result = [0]
    next_n = 1
    while next_n <= N:
        result.append(next_n)
        next_n = sum(result[-2:])
    return result
print(f"{fibonacci(0) = }")
print(f"{fibonacci(1) = }")
print(f"{fibonacci(50) = }")

fibonacci(0) = [0]
fibonacci(1) = [0, 1, 1]
fibonacci(50) = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


Configuramos la lista de resultados con un valor inicial de [0]. A continuación, iniciamos la iteración a partir del siguiente elemento $(next_n)$, que es 1. Mientras el siguiente elemento no sea mayor que $N$, seguimos añadiéndolo a la lista y calculando el siguiente valor de la secuencia. Calculamos el siguiente elemento tomando un trozo de los dos últimos elementos de la lista de resultados y pasándolo a la función $sum$.

Cuando la condición del bucle $while$ se evalúa como $False$, salimos del bucle y devolvemos el resultado. 

¿Y si sólo quisiera iterar sobre esos números? Podemos realizar el siguiente cambio en el código:

In [5]:
def fibonacci(N):
    """Return all fibonacci numbers up to N. """
    yield 0
    if N == 0:
        return
    a = 0
    b = 1
    while b <= N:
        yield b
        a, b = b, a + b
print(f"{list(fibonacci(0)) = }")
print(f"{list(fibonacci(1)) = }")
print(f"{list(fibonacci(50)) = }")

list(fibonacci(0)) = [0]
list(fibonacci(1)) = [0, 1, 1]
list(fibonacci(50)) = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


Ahora, la función $fibonacci()$ es una función generadora. Primero, obtenemos 0, y luego, si N es 0, devolvemos (esto provocará una excepción $StopIteration$). Si no es el caso, empezamos a iterar, produciendo $b$ en cada ciclo del bucle, y luego actualizando $a$ y $b$. Todo lo que necesitamos para poder producir el siguiente elemento de la secuencia son los dos pasados: $a$ y $b$, respectivamente.

Este código es mucho mejor, ocupa menos memoria, y todo lo que tenemos que hacer para obtener una lista de números Fibonacci es envolver la llamada con $list()$, como de costumbre. Podemos mejorar el código para que se vea más elegante de la siguiente forma:

In [6]:
def fibonacci(N):
    """Return all fibonacci numbers up to N. """
    a, b = 0, 1
    while a <= N:
        yield a
        a, b = b, a + b
print(f"{list(fibonacci(0)) = }")
print(f"{list(fibonacci(1)) = }")
print(f"{list(fibonacci(50)) = }")

list(fibonacci(0)) = [0]
list(fibonacci(1)) = [0, 1, 1]
list(fibonacci(50)) = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


Se puede observar, en este caso, que el uso de la asignación de tuplas $(a, b = 0, 1$ y $a, b = b, a + b)$ ayuda a hacer el código más corto y legible.